In [0]:
from google.colab import drive
drive.mount("/content/drive")
import os
os.chdir("/content/drive/My Drive/plant_pathology_kaggle")

##Preprocessing

In [0]:
import numpy as np
import pandas as pd

sam_sub = pd.read_csv('sample_submission.csv')
train_csv = pd.read_csv('train.csv')
test_csv = pd.read_csv('test.csv')

def get_class(row): 
  if row['multiple_diseases'] == 1: 
    return 'multiple_diseases' 
  elif row['rust'] == 1: 
    return 'rust' 
  elif row['scab'] == 1: 
    return 'scab' 
  else: 
    return 'healthy' 
    
train_csv["label"] = train_csv.apply(get_class, axis=1) 
#train_csv.head()
temp = pd.factorize(train_csv["label"])
train_csv["label"] = temp[0]

train_csv.head()

In [0]:
y = train_csv["label"]

from keras.utils import to_categorical
y = to_categorical(y, 4)

In [0]:
path = '/content/drive/My Drive/plant_pathology_kaggle/images/'
image_list = os.listdir(path)

train_images = train_csv["image_id"]
test_images = test_csv["image_id"]

train_images = np.asarray(train_images) + ".jpg"
test_images = np.asarray(test_images) + ".jpg"

In [0]:
image_height = 256
image_width = 256
channels = 3

import cv2
xtrain = np.zeros((len(y), image_height, image_width, channels))
xtest = np.zeros((len(y), image_height, image_width, channels))

for i in range(len(y)):
  imgtrain = train_images[i]
  imgtest = test_images[i]
  #path = '/content/drive/My Drive/plant_pathology_kaggle/images/' + img
  img = cv2.imread('/content/drive/My Drive/plant_pathology_kaggle/images/'+imgtrain)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (image_height, image_width))
  xtrain[i] = img
  img = cv2.imread('/content/drive/My Drive/plant_pathology_kaggle/images/'+imgtest)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (image_height, image_width))
  xtest[i] = img

xtrain /= 255.0
ytrain /= 255.0

In [0]:
from sklearn.model_selection import train_test_split
xtrain, xcv, ytrain, ycv = train_test_split(xtrain, y, test_size = 0.1, random_state = 267)

##Upsampling, none right now

## Models to try : (without upsampling, first)
i. EfficientNetB7
ii. DenseNet201
iii. VGG16
iv. ResNet
v. MobileNet


##Predictions

In [0]:
from keras.models import load_model
model = load_model("/content/saved.h5")

preds = model.predict(xtest)
#predcitions = np.argmax(predictions)